In [1]:
import pandas as pd

In [2]:
%pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [27]:
call_logs_df = pd.read_csv(r"C:\Users\Weaver 15\Documents\Capstone Project\AI-Assisted-Customer-Support-and-Appointment-Scheduling\CallLog_20240525-203319.csv")
transcriptions_df = pd.read_excel(r"C:\Users\Weaver 15\Documents\Capstone Project\AI-Assisted-Customer-Support-and-Appointment-Scheduling\Call_Recordings_Analysis (2).xlsx")
import re

In [12]:
transcriptions_df[['Date_Time', 'Identifier', 'Caller_ID', 'Call_Type', 'Unique_ID']] = transcriptions_df['file_name'].str.split('_', expand=True).iloc[:, :5]

# Merge DataFrames on "Phone Number"
merged_df = pd.merge(call_logs_df, transcriptions_df, how='left', left_on='Phone Number', right_on='Caller_ID')

# Drop redundant columns
merged_df.drop(columns=['file_name', 'Caller_ID'], inplace=True)

# Store merged DataFrame to Excel
merged_df.to_excel(r"C:\Users\Weaver 15\Documents\Capstone Project\AI-Assisted-Customer-Support-and-Appointment-Scheduling\merged_data.xlsx", index=True)

In [13]:
transcriptions_df

,file_name,transcription,duration,amplitude_mean,amplitude_max,amplitude_min,amplitude_std,total_silence_duration,sentiment_polarity,sentiment_subjectivity,Date_Time,Identifier,Caller_ID,Call_Type,Unique_ID
0,20240522-172226_2207_+19059665655_Outgoing_Aut...,"Hi there, you've reached",3.893878,-0.000024,0.865850,-0.768604,0.098242,3.127619,0.000000,0.000000,20240522-172226,2207,+19059665655,Outgoing,Auto
1,20240522-171853_2209_+16475218335_Outgoing_Aut...,"Hello, my speaking, am I speaking with Ms. Ma...",184.469887,-0.000100,0.889776,-0.949314,0.055745,90.939909,0.236296,0.455686,20240522-171853,2209,+16475218335,Outgoing,Auto
2,20240522-171921_2209_+19055025675_Outgoing_Aut...,"Hello, my speaking Jamil van Zing. Your call ...",6.125896,0.000041,0.954569,-0.939353,0.121890,1.416417,0.000000,0.000000,20240522-171921,2209,+19055025675,Outgoing,Auto
3,20240522-171921_2207_+16139697529_Outgoing_Aut...,"Hello. How am I speaking to Ruby? Yes. Hi, th...",62.357914,-0.000053,0.569082,-0.789197,0.049345,33.959909,0.148333,0.523333,20240522-171921,2207,+16139697529,Outgoing,Auto
4,20240522-172045_2207_+14164174030_Outgoing_Aut...,You have reached the voicemail box of 416.,5.405896,-0.000086,0.651359,-0.876268,0.090557,2.758821,0.000000,0.000000,20240522-172045,2207,+14164174030,Outgoing,Auto
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,20240522-165320_2207_+16479961057_Outgoing_Aut...,Your call has been forwarded to voice.,1.589887,-0.000125,0.480414,-0.365998,0.089016,0.127029,0.000000,0.000000,20240522-165320,2207,+16479961057,Outgoing,Auto
96,20240522-165611_2207_+16479380054_Outgoing_Aut...,"Hello. How am I singing to Sikun? Hi, this is...",151.565896,-0.000028,0.626477,-0.878704,0.054411,72.339410,0.032284,0.251632,20240522-165611,2207,+16479380054,Outgoing,Auto
97,20240522-164507_2209_+16478036954_Outgoing_Aut...,percent. Here it go. Hello. How July 28 of th...,23.981905,0.000009,0.751271,-0.963548,0.064520,10.235692,0.175000,0.566667,20240522-164507,2209,+16478036954,Outgoing,Auto
98,20240522-161434_+14168826025_Incoming_Auto_219...,"This call is being recorded. So, good afterno...",401.981905,-0.000044,0.967393,-1.026912,0.082868,190.239138,0.158934,0.468827,20240522-161434,+14168826025,Incoming,Auto,2197873601043.mp3


In [31]:
import pandas as pd
import re
from datetime import datetime

call_logs_df = call_logs_df[call_logs_df['Action Result']=='Call connected']
# Function to parse filename in table 2
def parse_filename(filename):
    match = re.match(r'(\d{8})-(\d{6})_\d+_\+(\d+)_.*', filename)
    if match:
        date_str, time_str, phone_number = match.groups()
        date = datetime.strptime(date_str, '%Y%m%d').strftime('%a %Y-%m-%d')
        time = datetime.strptime(time_str, '%H%M%S').strftime('%I:%M %p').lstrip('0').replace(' 0', ' ')
        phone_number = f"({phone_number[:3]}) {phone_number[3:6]}-{phone_number[6:]}"
        return date, time, phone_number
    return None, None, None

# Apply the parsing function to the file_name column in table 2
parsed_data = transcriptions_df['file_name'].apply(parse_filename)
transcriptions_df[['Date', 'Time', 'Phone Number']] = pd.DataFrame(parsed_data.tolist(), index=transcriptions_df.index)

# Merge the tables on Date, Time, and Phone Number
merged_table = pd.merge(call_logs_df, transcriptions_df, on=['Date', 'Time', 'Phone Number'], how='inner')

# Save the merged table to a new CSV file
merged_table.to_excel(r"C:\Users\Weaver 15\Documents\Capstone Project\AI-Assisted-Customer-Support-and-Appointment-Scheduling\merged_data.xlsx", index=True)

# Display the merged table
print(merged_table)


Empty DataFrame
Columns: [Type, Phone Number, Name, Date, Time, Action, Action Result, Result Description, Duration, file_name, transcription, duration, amplitude_mean, amplitude_max, amplitude_min, amplitude_std, total_silence_duration, sentiment_polarity, sentiment_subjectivity]
Index: []


In [38]:
transcriptions_df.sort_values(by='Phone Number')

,file_name,transcription,duration,amplitude_mean,amplitude_max,amplitude_min,amplitude_std,total_silence_duration,sentiment_polarity,sentiment_subjectivity,Date,Time,Phone Number
65,20240522-172132_2207_+12262246587_Outgoing_Aut...,You've reached fair flight.,3.389887,0.000012,0.162023,-0.150451,0.017975,2.507528,0.700000,0.900000,Wed 2024-05-22,5:21 PM,(122) 622-46587
68,20240522-172143_2207_+12262246587_Outgoing_Aut...,You've reached Fairfly. Leave a message please.,4.757914,0.000034,0.164575,-0.150381,0.016466,3.039637,0.000000,0.000000,Wed 2024-05-22,5:21 PM,(122) 622-46587
58,20240523-140536_2207_+12266277577_Outgoing_Aut...,Hello. Oh my speaking to my old. I'm asked is...,964.373878,-0.000064,1.013213,-1.035866,0.085535,491.267302,0.147525,0.498231,Thu 2024-05-23,2:05 PM,(122) 662-77577
41,20240523-181415_2282_+12268027772_Outgoing_Aut...,"Hello. Hey, hello. Am I speaking with Deb? Ye...",294.197914,-0.000066,1.071132,-1.165476,0.050018,198.276281,0.527461,0.742107,Thu 2024-05-23,6:14 PM,(122) 680-27772
50,20240523-170459_2208_+12269261051_Outgoing_Aut...,"Hello. Hi there. My name is Kriya Smada. Yes,...",763.637914,0.000049,1.171355,-1.169564,0.061563,397.691429,0.274403,0.525899,Thu 2024-05-23,5:04 PM,(122) 692-61051
...,...,...,...,...,...,...,...,...,...,...,...,...,...
38,20240523-182720_+13433631219_Incoming_Auto_219...,"This call is being recorded. Home, I speak in...",651.677914,-0.000219,0.948534,-0.960155,0.088083,326.505669,0.293219,0.566075,None,None,None
42,20240524-105552_+19055159568_Incoming_Auto_219...,This call is being recorded. Good morning. Th...,363.101905,-0.000308,0.927720,-0.991922,0.069033,258.147710,0.398052,0.484836,None,None,None
53,20240523-145659_+19054044662_Incoming_Auto_219...,"This call is being recorded. Hello, I speak i...",304.277914,-0.000259,0.946264,-0.970924,0.104566,139.625215,0.320775,0.501975,None,None,None
62,20240523-115616_+15192716476_Incoming_Auto_219...,This call is being recorded. Thank you. Thank...,437.981905,-0.000320,0.960735,-0.960155,0.044420,311.270612,0.223349,0.531484,None,None,None


In [5]:
import glob 

dir_list = glob.glob("Call_Recordings*")

In [6]:
dir_list

[]

In [7]:
import os

# Get the current working directory
current_directory = os.getcwd()

# Print the current working directory
print(current_directory)


C:\Windows\system32
